In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='talk', style='ticks',
        color_codes=True, rc={'legend.frameon': False})

%matplotlib inline

In [2]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
import nfp

In [3]:
import sys
sys.path.append('..')

In [4]:
from alphazero.preprocessor import preprocessor

In [5]:
import sys
sys.path.append('/projects/rlmolecule/pstjohn/spin_gnn/20200901_combined_losses/')
from loss import AtomInfMask, KLWithLogits

In [6]:
from nfp import EdgeUpdate, NodeUpdate, GlobalUpdate
import tensorflow_addons as tfa

model = tf.keras.models.load_model(
    '/projects/rlmolecule/svss/Project-Redox/water-model-tfrecords/20201013_stability_water_noh/best_model.hdf5',
    custom_objects={**nfp.custom_objects,
                    **{'AtomInfMask': AtomInfMask, 'KLWithLogits': KLWithLogits,
                       'EdgeUpdate': EdgeUpdate,
                       'NodeUpdate': NodeUpdate,
                       'GlobalUpdate': GlobalUpdate}})

In [82]:
new_model = tf.keras.Model(model.inputs[:3], [tf.nn.softmax(model.outputs[0]), model.outputs[1]])

In [98]:
shrees_df = pd.read_csv('/projects/rlmolecule/svss/Project-Redox/water-model-tfrecords/csv-spin-bv/buried_volumes_all_water_calc_includ_set_2.csv.gz', index_col=0)
shrees_spin = pd.read_csv('/projects/rlmolecule/svss/Project-Redox/water-model-tfrecords/csv-spin-bv/spins_water_calcs.csv')

shrees_df.head()

,atom_index,buried_vol,smiles
0,0,30.616607,CC[C@H]1[C]=CC[C@@H]1C
1,1,42.641684,CC[C@H]1[C]=CC[C@@H]1C
2,2,54.450699,CC[C@H]1[C]=CC[C@@H]1C
3,3,45.254120,CC[C@H]1[C]=CC[C@@H]1C
4,4,37.139451,CC[C@H]1[C]=CC[C@@H]1C


In [100]:
smiles = '[CH2]C1=COCC1'

In [101]:
mol = rdkit.Chem.MolFromSmiles(smiles)

In [104]:
preprocessor.construct_feature_matrices(mol, train=False)

{'atom': array([ 9, 58, 60, 57, 32, 32]),
 'bond': array([ 3,  3, 42, 17, 42, 39, 40, 40, 39, 17, 17, 17]),
 'connectivity': array([[0, 1],
        [1, 0],
        [1, 2],
        [1, 5],
        [2, 1],
        [2, 3],
        [3, 2],
        [3, 4],
        [4, 3],
        [4, 5],
        [5, 4],
        [5, 1]])}

In [106]:
new_model({key: tf.constant(np.expand_dims(val, 0))
       for key, val in preprocessor.construct_feature_matrices(mol, train=False).items()})

[<tf.Tensor: shape=(1, 6), dtype=float32, numpy=
 array([[0.38121492, 0.13322979, 0.4150315 , 0.05832914, 0.00418409,
         0.00801053]], dtype=float32)>,
 <tf.Tensor: shape=(1, 6, 1), dtype=float32, numpy=
 array([[[30.25468 ],
         [41.854122],
         [38.36191 ],
         [33.7179  ],
         [34.16185 ],
         [39.28588 ]]], dtype=float32)>]

In [110]:
shrees_df[shrees_df.smiles == smiles]

,atom_index,buried_vol,smiles
651259,0,30.291542,[CH2]C1=COCC1
651260,1,41.879991,[CH2]C1=COCC1
651261,2,38.252099,[CH2]C1=COCC1
651262,3,33.700549,[CH2]C1=COCC1
651263,4,34.074425,[CH2]C1=COCC1
651264,5,39.204603,[CH2]C1=COCC1


In [115]:
spin = shrees_spin[(shrees_spin.smiles == smiles) & (shrees_spin.atom_type != 'H')].spin

In [116]:
spin.abs() / spin.abs().sum()

0    0.405000
1    0.124902
2    0.393790
3    0.066754
4    0.001770
5    0.007784
Name: spin, dtype: float64

In [89]:
# import rdkit

# valid_dataset = tf.data.Dataset.from_generator(
#     lambda: (preprocessor.construct_feature_matrices(rdkit.Chem.MolFromSmiles(smiles), train=False) for smiles in valid),
#     output_types={'atom': tf.int64, 'bond': tf.int64, 'connectivity': tf.int64},
#     output_shapes={'atom': tf.TensorShape([None]), 'bond': tf.TensorShape([None]), 'connectivity': tf.TensorShape([None, 2])})\
#     .padded_batch(batch_size=100, padded_shapes={
#     'atom': [-1],
#     'bond': [-1],
#     'connectivity': [-1, 2]},
#     padding_values=nfp.zero)

In [92]:
predicted_spin_logits, predicted_bur_vol = model.predict(valid_dataset, verbose=1)

50/50 [==============================] - 3s 56ms/step


In [ ]:
cdf_test_pivot_spins = cdf_test.pivot(index='smiles', columns='atom_index', values='fractional_spin')
cdf_test_pivot_spins = cdf_test_pivot_spins.reindex(test)

In [117]:
new_model.save('/projects/rlmolecule/pstjohn/models/20201020_radical_stability_model/', include_optimizer=False)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /projects/rlmolecule/pstjohn/models/20201020_radical_stability_model/assets


In [ ]:
# model.save('/projects/rlmolecule/pstjohn/spin_gnn/test_tf_save', include_optimizer=False)